(containers)=
# Docker Containers

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/ok-transformer/blob/master/docs/nb/notes/containers.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/ok-transformer?style=social)](https://github.com/particle1331/ok-transformer)

---

**Readings.** [[Docker Guide]](https://docs.docker.com/language/python/)

## Introduction

Containerization solves the problem of running applications having multiple dependencies on the same machine, as well as having running applications consistently across different machines. A container runs in its isolated environments. Moreover, containers can be easily pulled by other machines allowing for easy installation, testing, and collaboration. We will use [Docker 🐳](https://www.docker.com/) which provides an ecosystem for efficiently working with containers. 

### Hello world

The following example demonstrates building and running a container:

In [1]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally


latest: Pulling from library/hello-world

Digest: sha256:ac69084025c660510933cca701f615283cdbb3aa0963188770b54c31c8962493
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



The above message tells the entire process of how the `hello-world` container eventually is able to run on our machine. The image was pulled on [Docker Hub](https://hub.docker.com/) which is a registry of Docker images. Note that the creation of images occurs locally since the local machine is also our compute layer. 

The container proceeds to run its default command, i.e. execute the `/hello` program that prints the message on the terminal. The `hello-world` image produces a minimal container whose sole purpose is to print this message.

```{figure} containers/img/00-helloworld.svg
---
name: helloworld
width: 600px
---
Anatomy of a Docker image and the resulting `hello-world` container in the context of the Linux kernel. Note the specific partition on the hard disk for the filesystem of the image.  
```

An **image** is essentially a filesystem snapshot with startup commands. This can be thought of as a read-only template which provides the daemon a set of instructions for creating a container. A **container** on the other hand is a running process in the Linux VM with partitioned hardware resources allocated by the kernel.

**Remark.** It would be significantly faster to run the `hello-world` container a second time since Docker uses a **cache** to build it. This makes sense since multiple containers are usually created from the same image.

### Interactive mode

As mentioned, containers have **isolated filesystems** by default. This means we can blow up a container and just create a fresh healthy container from the same image. This also ensures that our running processes will not affect the host computer which can be running other important processes. Running an [ubuntu](https://hub.docker.com/_/ubuntu) container in **detached** (`-d`) and **interactive mode** (`-it`): 

In [2]:
!docker run -d -it --name ubuntu0 ubuntu

Unable to find image 'ubuntu:latest' locally


latest: Pulling from library/ubuntu

2837585d: Pull complete .36MB/27.36MBBDigest: sha256:6042500cf4b44023ea1894effe7890666b0c5c7871ed83a97c36c76ae560bb9b
Status: Downloaded newer image for ubuntu:latest
f13b3ac5ea6f8f7aff60dc5081ccc3e14ea709b741600cab52e95f6d8854f355


This allows us to use the CLI inside the container using `docker exec`:

In [3]:
!docker exec ubuntu0 ls -C
!docker exec ubuntu0 rm -rf bin/ls
!docker exec ubuntu0 ls -C
!docker stop ubuntu0 > /dev/null

bin   dev  home  media	opt   root  sbin  sys  usr
boot  etc  lib	 mnt	proc  run   srv   tmp  var
OCI runtime exec failed: exec failed: unable to start container process: exec: "ls": executable file not found in $PATH: unknown


Creating a fresh container that can run `ls`. Note that the container ID is different:

In [4]:
!docker run -d -it --name ubuntu1 ubuntu
!docker exec ubuntu1 ls -C
!docker stop ubuntu1 > /dev/null

599dfa12c9ed0562c13ada95204314214d2eda8921d926665674fa32c393363e
bin   dev  home  media	opt   root  sbin  sys  usr
boot  etc  lib	 mnt	proc  run   srv   tmp  var


### Other commands

Listing all containers and images:

In [5]:
!docker ps --all

CONTAINER ID   IMAGE         COMMAND       CREATED          STATUS                                PORTS     NAMES
599dfa12c9ed   ubuntu        "/bin/bash"   11 seconds ago   Exited (137) Less than a second ago             ubuntu1
f13b3ac5ea6f   ubuntu        "/bin/bash"   23 seconds ago   Exited (137) 11 seconds ago                     ubuntu0
3ec84a52b2ce   hello-world   "/hello"      39 seconds ago   Exited (0) 38 seconds ago                       goofy_swartz


In [6]:
!docker image ls

REPOSITORY    TAG       IMAGE ID       CREATED        SIZE
ubuntu        latest    da935f064913   2 weeks ago    69.3MB
hello-world   latest    ee301c921b8a   7 months ago   9.14kB


**Remark.** The `hello-world` container immediately exited after running with exit status zero since no errors were encountered. The other containers continue running since they are run in interactive mode. 

To stop containers, we can use either `stop` or `kill`. The `stop` command sends a SIGTERM to the running process. This gives 10 seconds for cleanup, then a fallback SIGKILL is sent to immediately terminate the process. See {numref}`lifecycle` and [restart policies](https://docs.docker.com/engine/reference/run/#restart-policies---restart) docs.

```{figure} containers/img/00-lifecycle.png
---
name: lifecycle
---
Complete Docker container lifecycle. [[source](https://docker-saigon.github.io/post/Docker-Internals/)]

```

## Docker build

Throughout the above examples we have been using public images from Docker Hub. 
In this section, we create our own images for running our own containers. Our custom images can be pushed to container
repositories, such as Docker Hub or [ECR](https://aws.amazon.com/ecr/), which our servers can pull 
to run our containers remotely. To do this, Docker requires us to create a `Dockerfile` which specifies the container build process.

In [7]:
!tree ./containers/simple-fastapi -I __pycache__

./containers/simple-fastapi
├── Dockerfile
├── requirements.txt
└── src
    └── main.py

2 directories, 3 files


The web app simply prints a message when the root URI is called:

In [8]:
!pygmentize ./containers/simple-fastapi/src/main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello world!"}


### Dockerfile

The following `Dockerfile` uses `python:3.10-slim` as **base image**. This `slim` image is a smaller version of a container running Python 3.10, but still larger than `alpine`. The next lines serve to modify the base image. First, it specifies `/code` as the **working directory**. This is where all subsequent build commands will be executed. 

The copy command copies files in the build folder to a path relative to the working directory. Next, we call `pip` with certain flags so that it does not cache the installs, making the container more minimal. The final line `CMD` specifies the command executed when running the container. See also [`ENTRYPOINT`](https://spacelift.io/blog/docker-entrypoint-vs-cmd).

In [9]:
!pygmentize ./containers/simple-fastapi/Dockerfile

FROM python:3.10-slim

WORKDIR /code

COPY ./requirements.txt ./
RUN pip install --no-cache-dir --upgrade -r requirements.txt

COPY ./src ./src

CMD ["uvicorn", "src.main:app", "--host", "0.0.0.0", "--port", "80", "--reload"]


Building the image, we also add a **path** (this defaults to the `latest` [tag](https://docs.docker.com/engine/reference/commandline/tag/)):

In [10]:
!docker build ./containers/simple-fastapi -t okt/simple-fastapi

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

### Cached layers

The order of Dockerfile instructions matters. Each instruction in a Dockerfile roughly translates to an **image layer**. The layers are cached in the build process. Hence, changes in one layer destroys the cache of subsequent layers. This is known as **cache busting**. This explains why requirements are installed first in the `Dockerfile` before source code is copied, so that modifying the source does not result in reinstalling the dependencies ({numref}`cache`).

The following created timestamps indicate cached layers in our previous build:

In [11]:
!docker history okt/simple-fastapi

IMAGE          CREATED                  CREATED BY                                      SIZE      COMMENT
db97ac771b67   Less than a second ago   CMD ["uvicorn" "src.main:app" "--host" "0.0.…   0B        buildkit.dockerfile.v0
<missing>      Less than a second ago   COPY ./src ./src # buildkit                     425B      buildkit.dockerfile.v0
<missing>      About an hour ago        RUN /bin/sh -c pip install --no-cache-dir --…   20.7MB    buildkit.dockerfile.v0
<missing>      About an hour ago        COPY ./requirements.txt ./ # buildkit           31B       buildkit.dockerfile.v0
<missing>      19 hours ago             WORKDIR /code                                   0B        buildkit.dockerfile.v0
<missing>      2 months ago             CMD ["python3"]                                 0B        buildkit.dockerfile.v0
<missing>      2 months ago             RUN /bin/sh -c set -eux;   savedAptMark="$(a…   12.2MB    buildkit.dockerfile.v0
<missing>      2 months ago             ENV PYT

<br>

```{figure} containers/img/layers.png
---
name: layers
width: 600px
---
Dockerfile translates into a stack of layers in a container image. [Source](https://docs.docker.com/build/guide/layers/)
```

```{figure} containers/img/00-cache-busting.svg
---
name: cache
width: 600px
---
Busting an expensive cached layer (left). Cache optimized version (right).
```

Listing the built image:

In [12]:
!docker image ls

REPOSITORY           TAG       IMAGE ID       CREATED        SIZE
okt/simple-fastapi   latest    db97ac771b67   1 second ago   175MB
ubuntu               latest    da935f064913   2 weeks ago    69.3MB
hello-world          latest    ee301c921b8a   7 months ago   9.14kB


### Port mapping

Note that the app runs in port `0.0.0.0:80` inside the container. We will expose this to our local machine by **port mapping** it to `localhost:8000`. Running the image in detached mode:

In [13]:
!docker run -d -p 8000:80 --name fastapi okt/simple-fastapi:latest

5a9d508c584aadea2a01394063174d8d65f39b20a29520ec7b44c04c218bb53c


In [14]:
!docker ps

CONTAINER ID   IMAGE                       COMMAND                  CREATED        STATUS                  PORTS                  NAMES
5a9d508c584a   okt/simple-fastapi:latest   "uvicorn src.main:ap…"   1 second ago   Up Less than a second   0.0.0.0:8000->80/tcp   fastapi


In [15]:
import time
time.sleep(3)

Trying it out:

In [16]:
!http :8000

HTTP/1.1 200 OK
content-length: 26
content-type: application/json
date: Wed, 27 Dec 2023 17:30:47 GMT
server: uvicorn

{
    "message": "Hello world!"
}




## Dev environment

This section deals with quality of life improvement for developing with Docker. For example, any changes to `main.py` will not affect the running app (which is isolated). This can make development difficult with time-consuming rebuilds. Also, if we have a large dataset that will be used for development, then we will have to copy this every time the image is built. Our IDE also does not have things like autocomplete and will typically compain of missing packages.

```{figure} ./containers/img/vs-code-module-not-found.png
---
name: vs-code-module-not-found
width: 600px
---
Module not found. One unsatisfying solution is to install the requirements file in a virtual environment. But this is not ideal if you want strict reproducibility.
```

### Volumes

The issue with code changes and data is fixed by using **volumes**. This will allow changes in the local filesystem to be reflected within the container (since files are mirrored between the two directories). In our case, the `--reload` flag is essential in order to avoid manually restarting the uvicorn server inside the container. But this has to be removed in the production `Dockerfile`.

**Remark.** Note that we can have `Dockerfile.dev` and `Dockerfile`. We just need to specify the specific file when using `docker build`.

In [17]:
!tree $(pwd)/containers/simple-fastapi -I __pycache__

/Users/particle1331/code/ok-transformer/docs/nb/notes/containers/simple-fastapi
├── Dockerfile
├── requirements.txt
└── src
    └── main.py

2 directories, 3 files


Running `docker run` with `-v` flag for mapping volumes:

In [18]:
!docker rm -f fastapi >> /dev/null  # delete prev container
!docker run -d -p 8000:80 -v $(pwd)/containers/simple-fastapi:/code --name fastapi okt/simple-fastapi:latest

d4c13142e2dc79371978a6c579ab3f5ef0f7dc56573ebdfcc8e3cb8e3820f84b


Modifying the main file:

In [19]:
%%writefile ./containers/simple-fastapi/src/main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello world + 123!"}    # (!)

Overwriting ./containers/simple-fastapi/src/main.py


See if output changes without rebuild (this uses [httpie](https://httpie.io/docs/cli/universal)):

In [20]:
import time
time.sleep(3)

In [21]:
!http :8000

HTTP/1.1 200 OK
content-length: 32
content-type: application/json
date: Wed, 27 Dec 2023 17:30:53 GMT
server: uvicorn

{
    "message": "Hello world + 123!"
}




In [22]:
%%writefile ./containers/simple-fastapi/src/main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello world!"}

Overwriting ./containers/simple-fastapi/src/main.py


### IDE for a running container

To follow this section, you have to install [Docker](https://marketplace.visualstudio.com/items?itemName=ms-azuretools.vscode-docker) and [Dev Containers](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-containers) extensions in VS Code. To use an IDE with a running container, you can click on the lower left button or press CTRL+SHIFT+P and type "Dev Containers: Attach to running container". This opens up a new window. You have to install the [Python extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) once to get IDE features. The experience is the same as when you SSH into a remote server. 

```{figure} ./containers/img/vs-code-completion.png
---
name: vs-code-completion
width: 1200px
---
IDE attached to the container with code completion and other useful features.
```

In [23]:
!docker rm -f fastapi >> /dev/null

### Debugging

***Note:** This section should be followed while the compose application below discussed in the next section is running.*

For debugging, we use https://github.com/microsoft/debugpy. After installing the Python extension, go to the Debug tab in VS Code. Then click "create a launch.json file". In the options, select "Remote Attach" and enter "localhost" with port "5678". We need to integrate the following code in the main file:

```python
import debugpy

debugpy.listen(("0.0.0.0", 5678))
print("Waiting for client to attach...")
debugpy.wait_for_client()
```

After adding breakpoints, we can start running the debugger {numref}`vs-code-debugger` and debugging a request.

```{figure} ./containers/img/vs-code-debugger.png
---
name: vs-code-debugger
width: 1200px
---
Debugger running. Breakpoints are triggered after a GET request to `localhost:8080`.
```

## Docker compose

Multi-container applications require configuring setup and tear down of run and builds, volumes, as well as networking between multiple services. This can be tedious to do using Docker CLI commands especially during development. [Docker Compose](https://docs.docker.com/compose/) allows us to collect all configurations in a YAML file. This takes care of networking between containers as well as logging and status monitors for the whole ensemble.

### Files

The web server is in its own directory containing its corresponding Dockerfile:

In [24]:
!tree containers/compose -I __pycache__

containers/compose
├── app
│   ├── Dockerfile
│   ├── requirements.txt
│   └── src
│       └── main.py
└── docker-compose.yml

3 directories, 4 files


This simply tracks the visit count along with a message. For storing the counts, we will use a [redis](https://redis.io/docs/connect/clients/python/) database. Note that since compose takes care of networking, it suffices to use the container name (see compose file below) as host for the redis client:

In [25]:
!pygmentize containers/compose/app/src/main.py

import redis
from fastapi import FastAPI

app = FastAPI()
r = redis.Redis(host="redis", port=6379)

if not r.exists("visits"):
    r.set("visits", 0)


@app.get("/")
def root():
    visits = int(r.get("visits")) + 1
    r.set("visits", visits)
    return {
        "message": "Hello world!",
        "visit_count": r.get("visits"),
    }


Note that paths in Dockerfile and compose files are relative:

In [26]:
!pygmentize containers/compose/app/Dockerfile

FROM python:3.10-slim

WORKDIR /code

COPY ./requirements.txt ./

RUN pip install --no-cache-dir --upgrade -r requirements.txt

COPY ./src ./src

ENTRYPOINT ["uvicorn", "src.main:app"]


Observe that we use `ENTRYPOINT` here with arguments specified in the ff. compose file:

In [27]:
!pygmentize containers/compose/docker-compose.yml

version: '3'
services:
  fastapi-server:
    build: app
    command: '--host 0.0.0.0 --port 80 --reload'
    restart: on-failure
    ports:
      - 8080:80
      - 5678:5678
    volumes:
      - ./app:/code
    depends_on:
      - redis

  redis:
    image: redis:alpine


### Compose up

Starting the multi-container application. The `-f` to point to the compose file. If we are in the directory, then it suffices to call `docker compose up`. The `--build` flag is used to rebuild containers from the images. Again, we use `-d` to run it in detached mode.

In [28]:
!docker-compose -f "./containers/compose/docker-compose.yml" up -d --build

[+] Running 0/0
 ⠋ redis Pulling                                                           0.1s
[+] Running 0/1
 ⠙ redis Pulling                                                           0.2s
[+] Running 0/1
 ⠹ redis Pulling                                                           0.3s
[+] Running 0/1
 ⠸ redis Pulling                                                           0.4s
[+] Running 0/1
 ⠼ redis Pulling                                                           0.5s
[+] Running 0/1
 ⠴ redis Pulling                                                           0.6s
[+] Running 0/1
 ⠦ redis Pulling                                                           0.7s
[+] Running 0/1
 ⠧ redis Pulling                                                           0.8s
[+] Running 0/1
 ⠇ redis Pulling                                                           0.9s
[+] Running 0/1
 ⠏ redis Pulling                                                           1.0s
[+] Running 0/1
 ⠋ redis Pulling        

**Remark.** Services which are still up will not be rebuilt. As such, these services can persist data and state while rebuilt services have reset state.

In [29]:
!docker compose -f "./containers/compose/docker-compose.yml" ps

NAME                       IMAGE                    COMMAND                  SERVICE             CREATED             STATUS                  PORTS
compose-fastapi-server-1   compose-fastapi-server   "uvicorn src.main:ap…"   fastapi-server      1 second ago        Up Less than a second   0.0.0.0:5678->5678/tcp, 0.0.0.0:8080->80/tcp
compose-redis-1            redis:alpine             "docker-entrypoint.s…"   redis               1 second ago        Up Less than a second   6379/tcp


In [30]:
time.sleep(3)

Making a request:

In [31]:
!http :8080

HTTP/1.1 200 OK
content-length: 44
content-type: application/json
date: Wed, 27 Dec 2023 17:31:15 GMT
server: uvicorn

{
    "message": "Hello world!",
    "visit_count": "1"
}




In [32]:
!http :8080

HTTP/1.1 200 OK
content-length: 44
content-type: application/json
date: Wed, 27 Dec 2023 17:31:16 GMT
server: uvicorn

{
    "message": "Hello world!",
    "visit_count": "2"
}




Monitoring running services resource usage:

In [33]:
!docker stats --no-stream $(docker ps --format "{{.Names}}" | grep -w 'compose')

CONTAINER ID   NAME                       CPU %     MEM USAGE / LIMIT     MEM %     NET I/O           BLOCK I/O    PIDS
e975c3e4023e   compose-fastapi-server-1   1.35%     54.91MiB / 3.841GiB   1.40%     3.57kB / 2.89kB   0B / 197kB   4
86f5c9df191d   compose-redis-1            0.37%     2.812MiB / 3.841GiB   0.07%     2.97kB / 1.36kB   0B / 0B      5


Teardown:

In [34]:
!docker compose -f "./containers/compose/docker-compose.yml" down

[+] Running 0/0
 ⠋ Container compose-fastapi-server-1  Stopping                            0.1s
[+] Running 0/1
 ⠙ Container compose-fastapi-server-1  Stopping                            0.2s
[+] Running 0/1
 ⠹ Container compose-fastapi-server-1  Stopping                            0.3s
[+] Running 0/1
 ⠸ Container compose-fastapi-server-1  Stopping                            0.4s
[+] Running 0/1
 ⠼ Container compose-fastapi-server-1  Stopping                            0.5s
[+] Running 1/1
 ⠿ Container compose-fastapi-server-1  Removed                             0.5s
 ⠋ Container compose-redis-1           Stop...                             0.1s
[+] Running 1/2
 ⠿ Container compose-fastapi-server-1  Removed                             0.5s
 ⠙ Container compose-redis-1           Stop...                             0.2s
[+] Running 2/2
 ⠿ Container compose-fastapi-server-1  Removed                             0.5s
 ⠿ Container compose-redis-1           Remo...                          

## Readings: Best practices

* [What is the best way to pass AWS credentials to a Docker container?](https://stackoverflow.com/a/76191745/1091950)
* [Security best practices](https://docs.docker.com/develop/security-best-practices/)
* [Optimizing builds with cache management](https://docs.docker.com/build/cache/)
* [Docker Best Practices for Python Developers](https://testdriven.io/blog/docker-best-practices/)